In [1]:
%cd /home/tk/repos/erc

/home/tk/repos/erc


In [2]:
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification
from utils import ErcTextDataset
import os

In [3]:
OUTPUT_DIR = './results/IEMOCAP/roberta-base/2021-05-19-22-37-58/'
tokenizer = RobertaTokenizerFast.from_pretrained(os.path.join(OUTPUT_DIR, 'tokenizer'))

In [ ]:
DATASET = 'IEMOCAP'
num_past_utterances=1000
num_future_utterances=1000
ADD_BOU_EOU = True
ADD_SPEAKER_TOKENS=True
ROOT_DIR = './multimodal-datasets/'
SEED=42

if ADD_SPEAKER_TOKENS:
    ADD_SPEAKER_TOKENS = os.path.join(OUTPUT_DIR, 'tokenizer', 'added_tokens.json')


ds_train = ErcTextDataset(DATASET=DATASET, SPLIT='train', 
                          num_past_utterances=num_past_utterances, num_future_utterances=num_future_utterances,
                          model_checkpoint=os.path.join(OUTPUT_DIR, 'tokenizer'),
                          ADD_BOU_EOU=ADD_BOU_EOU, ADD_SPEAKER_TOKENS=ADD_SPEAKER_TOKENS,
                          ROOT_DIR=ROOT_DIR, SEED=SEED)

ds_val = ErcTextDataset(DATASET=DATASET, SPLIT='val', 
                        num_past_utterances=num_past_utterances, num_future_utterances=num_future_utterances,
                        model_checkpoint=os.path.join(OUTPUT_DIR, 'tokenizer'),
                        ADD_BOU_EOU=ADD_BOU_EOU, ADD_SPEAKER_TOKENS=ADD_SPEAKER_TOKENS,
                        ROOT_DIR=ROOT_DIR, SEED=SEED)

ds_test = ErcTextDataset(DATASET=DATASET, SPLIT='test',
                         num_past_utterances=num_past_utterances, num_future_utterances=num_future_utterances,
                         model_checkpoint=os.path.join(OUTPUT_DIR, 'tokenizer'),
                         ADD_BOU_EOU=ADD_BOU_EOU, ADD_SPEAKER_TOKENS=ADD_SPEAKER_TOKENS,
                         ROOT_DIR=ROOT_DIR, SEED=SEED)

2021-05-19 22:38:21.315 INFO utils - _load_utterance_ordered: every text file exists fine!
2021-05-19 22:38:21.316 INFO utils - _string2tokens: converting utterances into tokens ...
2021-05-19 22:38:21.317 INFO utils - _string2tokens: creating input utterance data ... 
100%|██████████| 100/100 [00:11<00:00,  8.74it/s]
2021-05-19 22:38:32.799 INFO utils - _create_input: number of truncated utterances: 4760
2021-05-19 22:38:32.837 INFO utils - _load_utterance_ordered: every text file exists fine!
2021-05-19 22:38:32.838 INFO utils - _string2tokens: converting utterances into tokens ...
2021-05-19 22:38:32.839 INFO utils - _string2tokens: creating input utterance data ... 
100%|██████████| 20/20 [00:02<00:00,  8.96it/s]
2021-05-19 22:38:35.126 INFO utils - _create_input: number of truncated utterances: 980
2021-05-19 22:38:35.181 INFO utils - _load_utterance_ordered: every text file exists fine!
2021-05-19 22:38:35.182 INFO utils - _string2tokens: converting utterances into tokens ...
202

In [ ]:
import random
ds = ds_test
idx = random.randint(0, len(ds))

input_ids = ds[idx]['input_ids']
print(input_ids)
print()
print(tokenizer.decode(input_ids))

In [9]:
special_tokens_dict = {'additional_special_tokens': ['<u>', '</u>', '<stranger>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(50268, 1024)

In [10]:
len(tokenizer)

50268

In [179]:
tokenizer.additional_special_tokens

[]

In [168]:
len(tokenizer)

50268

In [169]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>',
 'additional_special_tokens': "['<u>', '</u>', '<stranger>']"}

In [170]:
tokenizer.additional_special_tokens_ids

[50265, 50266, 50267]

In [173]:
tokenizer('<s><u> </u>, <stranger>')

{'input_ids': [0, 0, 50265, 1437, 50266, 6, 1437, 50267, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [172]:
tokenizer('<s><u> </u>, stranger')

{'input_ids': [0, 0, 50265, 1437, 50266, 6, 12443, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [122]:
model.save_pretrained

Signature:
model.save_pretrained(
    save_directory: Union[str, os.PathLike],
    save_config: bool = True,
    state_dict: Union[dict, NoneType] = None,
    save_function: Callable = <function save at 0x7f1da364c4d0>,
    push_to_hub: bool = False,
    **kwargs,
)
Docstring:
Save a model and its configuration file to a directory, so that it can be re-loaded using the
`:func:`~transformers.PreTrainedModel.from_pretrained`` class method.

Arguments:
    save_directory (:obj:`str` or :obj:`os.PathLike`):
        Directory to which to save. Will be created if it doesn't exist.
    save_config (:obj:`bool`, `optional`, defaults to :obj:`True`):
        Whether or not to save the config of the model. Useful when in distributed training like TPUs and need
        to call this function on all processes. In this case, set :obj:`save_config=True` only on the main
        process to avoid race conditions.
    state_dict (nested dictionary of :obj:`torch.Tensor`):
        The state dictionary of

In [124]:
foo = model.from_pretrained('./')

In [126]:
from transformers import AutoTokenizer

In [129]:
tokenizer.save_pretrained('./')

('./tokenizer_config.json',
 './special_tokens_map.json',
 './vocab.json',
 './merges.txt',
 './added_tokens.json')

In [131]:
tokenizer = AutoTokenizer.from_pretrained('./')

In [134]:
tokenizer('</u>')

{'input_ids': [0, 50266, 2], 'attention_mask': [1, 1, 1]}